In [6]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
warnings.filterwarnings('ignore')
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
from natsort import natsorted
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [ ]:
customer_id = '5'
formatted_attribute = 'weight'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
df = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(df))

In [ ]:
customer_id = '5'
formatted_attribute = 'material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
df = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(df))

In [ ]:
df[df['value'].astype(str)=='Fabric']['buckets'].explode().value_counts()
fabric=df[df['value'].astype(str)=='Fabric']
fab=fabric[(fabric['buckets'].astype(str)=='Travel Pillows')|(fabric['buckets'].astype(str)=='Tablecloths & Table Runners')].sort_values('resolution')
print(len(fab))
fab[curation_col]='n/a'
match_fab=fab[['external_id',curation_col]]
match_fab

In [ ]:
polyester=df[(df['value'].astype(str)=='Polyester')&(df['buckets'].astype(str)=='Travel Pillows')]
print(len(polyester))
pat=r'(?i)(microplush)|(microfiber)|()'
polyester['matches']=polyester['long_desc'].apply(lambda x: re_extract(pat, x))
print(len(polyester[polyester['matches'].astype(str)!='[]']))
poly=polyester[polyester['matches'].astype(str)!='[]']
poly[curation_col]='n/a'
match_poly=poly[['external_id',curation_col]]
match_poly
poly

In [ ]:
blend=df[(df['value'].astype(str)=='Polyester')]
print(len(blend))
pat=r'(?i)(\d+(?<!100)\s?\%\spolyester)|()'
blend['matches']=blend['long_desc'].apply(lambda x: re_extract(pat, x))
print(len(blend[blend['matches'].astype(str)!='[]']))
bl=blend[blend['matches'].astype(str)!='[]']
bl[curation_col]='Blend'
match_bl=bl[['external_id',curation_col]]
match_bl
bl

In [ ]:
linen=df[(df['buckets'].astype(str)=='Tablecloths & Table Runners')]
pat=r'(?i)(.{0,20}linen.{0,20})|()'
linen['matches']=linen['long_desc'].apply(lambda x: re_extract(pat, x))

pat_value=r'(?i)(linen)|()'
linen['matches_value']=linen['value'].apply(lambda x: re_extract(pat_value, x))

In [ ]:
lin=linen[(linen['matches'].astype(str)!='[]')&(linen['matches_value'].astype(str)=='[]')]
lin['match']=lin['value'].apply(lambda x: re.sub(r'Burlap(?!")','["Burlap",Linen"]',str(x))).apply(lambda x: re.sub(r'Fabric(?!")','["Fabric","Linen"]',str(x))).apply(lambda x: re.sub(r'Natural Material(?!")','["Linen","Natural Material"]',str(x))).apply(lambda x: re.sub(r'n/a','Linen',str(x)))
lin['match']=lin['match'].apply(lambda x: re.sub(r'(?i)Leather(?!")','["Leather","Linen"]',str(x))).apply(lambda x: re.sub(r'(?i)Rubber(?!")','["Rubber","Linen"]',str(x))).apply(lambda x: re.sub(r'(?i)Polyester(?!")','["Linen","Polyester"]',str(x))).apply(lambda x: re.sub(r'(?i)Cotton(?!")','["Cotton","Linen"]',str(x)))#.apply(lambda x: re.sub(r'"Cotton(?=")','Cotton","Linen"',str())).apply(lambda x: re.sub(r'Cotton(?!")(?!"\,)(?!\"\])','["Cotton","Linen"]',str())).apply(lambda x: re.sub(r'Polyester(?!")(?!"\,)(?!\"\])','["Polyester","Linen"]',str())).apply(lambda x: re.sub(r'"Polyester(?=")','"Linen","Polyester',str()))                                         
print(len(lin))
lin['match'].explode().value_counts()
match_lin=lin[['external_id','match']]
match_lin

In [ ]:
stop

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_fab))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_poly))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_bl))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_lin))

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, match_fab)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, match_poly)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, match_bl)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, match_lin)

In [ ]:
err

# Weight

In [ ]:
customer_id = '5'
formatted_attribute = 'weight'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
df = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(df))

In [ ]:
regex_pattern = r"(?i)((?<!supports)(?<!minimum)(?<!light)(?<!max)(?<!maximum)(?<!maximum child)(?<!accommodates)\sweighs?(?!t\scapacity)(?!ing)\s?\d+\.?\d*\s?lb)|()"               
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, str(x)))
df['matches']=df['matches'].apply(lambda x: re.sub(r'\[\]','n/a',str(x))).apply(lambda x: re.sub(r'''(?i)Weighs?\s?|\['|'\]''','',str(x)))
matchdf=df[['attribute','external_id','name','long_desc','buckets','curated_date','matches']]
matchdf.to_csv('WeightML.csv')

In [ ]:
# for i in range(len(df)):
#     if df['matches'][i]!=df['value'][i]:
#         print(df['external_id'][i])
# # df[0:500]

# Diameter

In [ ]:
customer_id = '5'
formatted_attribute = 'diameter'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
df = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(df))

In [ ]:
df

In [ ]:
pat=r'''(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|(\d+\.?\d*.Inch Round)|()'''
# pat_name=r'''(?i)(\d+\.?\d*.Inch Round)|()'''
df['matches'] = df['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# df['matches_name'] = df['name'].apply(lambda x: re_extract(pat, str(x)))
df['matches']=df['matches'].apply(lambda x: re.sub(r'\[\]','n/a',str(x))).apply(lambda x: re.sub(r'''(?i)Diameter?\s?|\['|'\]''','',str(x))).apply(lambda x: re.sub(r'\\{0,3}"',' in',str(x))).apply(lambda x: re.sub(r"'\,'",',',str(x)))
df.to_csv('Data/diameterML')
# matchdf=df[['attribute','external_id','name','long_desc','buckets','curated_date','matches']]
# matchdf.to_csv('diameterML.csv')
# df[0:500]

# Length

In [ ]:
customer_id = '5'
formatted_attribute = 'length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
df = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(df))

In [ ]:
regex_pattern = r"(?i)(?<!cord.measures )(?<!capacity )((?<!\.)(?<!\-)(?<!\d)(?<!\,)\d{1,3}(?!\d)(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?e?s?)?(?:\Wft)?(?:\Wfoot)?)\WL(?:ength|ong(?! rows)(?! lasting)(?! sleeves))?\b)|((?:l(?:ength)?|long(?! rows)):\s?\d+\s?inc?h?e?s?)|(Le?n?g?t?h?\:\s?10\s?in)|((?<!\.)(?<!\/)\d+(?!\.).{0,3}\b(?:l(?:ength))\b)|()"               
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, str(x)))
df['matches']=df['matches'].apply(lambda x: re.sub(r'\[\]','n/a',str(x))).apply(lambda x: re.sub(r'''(?i)Weighs?\s?|\['|'\]''','',str(x)))
matchdf=df[['attribute','external_id','name','long_desc','buckets','curated_date','matches']]
matchdf.to_csv('LengthML.csv')

In [ ]:
err

In [ ]:


print(len(df))
df_none=df[df['buckets'].astype(str)=='None']
print(len(df_none))
df=df[df['buckets'].astype(str)!='None']
print(len(df))
df_na=df[df['value'].astype(str)=='n/a']
print(len(df_na))
df_new=df[df['value'].astype(str)!='n/a']
df.head(2)

In [ ]:
regex_pattern_length=r'(?i)\d\DLi?t?e?r?|()'         
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
df[df['matches'].astype(str)!='[]']

In [ ]:
# df[df['value'].astype(str)!='n/a']

In [ ]:
regex_pattern_length=r'(?i).{0,40}(water.?proof).{0,40}|()'         
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(df))
df_na=df[df['value'].astype(str)=='n/a']
print(len(df_na))
df_final=df_na[df_na['matches'].astype(str)!='[]']
print(len(df_final))

In [ ]:
pat='(?i)(^(?=.*?(?:water.?proof(?! liner not included)))(?:(?!never).)*$)|()'
df_final['matches'] = df_final['long_desc'].apply(lambda x: re_extract(pat, x))
print(len(df_final[df_final['matches'].astype(str)!='[]']))
df_proof=df_final[df_final['matches'].astype(str)!='[]']

In [ ]:
df_proof[curation_col]='Waterproof'
match_proof=df_proof[['external_id',curation_col]]
match_proof

In [ ]:
dataset=df_final
#Fetch wordcount for each abstract
dataset['word_count'] = dataset['long_desc'].apply(lambda x: len(str(x).split(" ")))
dataset[['long_desc','word_count']].head()

In [ ]:
##Descriptive statistics of word counts
dataset.word_count.describe()

In [ ]:
#Identify common words
freq = pd.Series(' '.join(dataset['long_desc']).split()).value_counts()[:20]
freq

In [ ]:
#Identify uncommon words
freq1 =  pd.Series(' '.join(dataset 
         ['long_desc']).split()).value_counts()[-20:]
freq1

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizerlem = WordNetLemmatizer()
stem = PorterStemmer()word = "inversely"print("stemming:",stem.stem(word))
print("lemmatization:", lem.lemmatize(word, "v"))

In [ ]:
# Libraries for text preprocessing
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))##Creating a list of custom stopwords
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown"]
stop_words = stop_words.union(new_words)

In [ ]:
corpus = []
for i in range(len(dataset['long_desc'])):
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', dataset['long_desc'][0:i])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500

cv = CountVectorizer(stop_words='english') 
cv_matrix = cv.fit_transform(df_final['long_desc'][0:100]) 
# create document term matrix
df_dtm = pd.DataFrame(cv_matrix.toarray(), columns=cv.get_feature_names())
df_dtm

In [ ]:
[*df_dtm][71:]

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# df_dtm.plot(kind='bar')

# plt.show()

In [ ]:
# cv = CountVectorizer(stop_words='english') 
# cv_matrix = cv.fit_transform(df_final['long_desc'][0:100]) 
# # create document term matrix
# df_dtm = pd.DataFrame(cv_matrix.toarray(), columns=cv.get_feature_names())
# df_dtm

In [ ]:
# df_final[0:500]

In [ ]:
regex_pattern_length=r'(?i)(^(?=.*?(?:(?<!use a )water.?resistant(?! liner not included)))(?:(?!never).)*$)|()'         
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(df))
df_na=df[df['value'].astype(str)=='n/a']
print(len(df_na))
df_final_resist=df_na[df_na['matches'].astype(str)!='[]']
print(len(df_final_resist))

In [ ]:
df_final_resist[curation_col]='Water Resistant'
match_resist=df_final_resist[['external_id',curation_col]]
match_resist

# Care

In [ ]:
customer_id = '5'
formatted_attribute = 'Care'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
#           'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
care = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)

print(len(care))
cares=care[care['buckets'].astype(str)=='None']
print(len(cares))
care=care[care['buckets'].astype(str)!='None']
care['bucket_id']=care['bucket_id'].astype(int)
print(len(care))
care_na=care[care['value'].astype(str)=='n/a']
print(len(care_na))
care.head(2)

In [ ]:
throws=care[care['buckets'].astype(str)=='Throw/Decorative Pillows']
# print(len(throws[throws['value'].astype(str)=='n/a']))
throw_na=throws[throws['value'].astype(str)=='n/a']
throw_na[curation_col]=''
match_throw_na=throw_na[['external_id',curation_col]]

In [ ]:
print(len(throws))
regex_pat=r'(?i)(tumble.?dry)|()'  
throws['matches'] = throws['long_desc'].apply(lambda x: re_extract(regex_pat, x))
throws['matches'].explode().value_counts()
throws[throws['matches'].astype(str)!='[]']

In [ ]:
# df[df['Percent N/A']>=0.9]['Bucket Id'].to_list()

In [ ]:
bucket_list=list(set(care['bucket_id']))
df = pd.DataFrame([bucket_list[i], len(care[care['bucket_id']==bucket_list[i]]), round(len(care_na[care_na['bucket_id']==bucket_list[i]])/len(care[care['bucket_id']==bucket_list[i]]),2)] for i in range(len(bucket_list)))
df.rename({0: "Bucket Id", 1: "Length", 2: "Percent N/A"}, axis='columns', inplace =True)
df.sort_values(by=['Percent N/A'], ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df[0:100]

In [ ]:
regex_pat=r'(?i)(professional upholstery)|(spot.?clean)|(vacuum)|(wipe.?clean)|(air.?dry)|(dry.?clean)|(spot.?clean)|()'  
care_na['done'] = care_na['long_desc'].apply(lambda x: re_extract(regex_pat, x))
print(len(care_na))
care_na=care_na[care_na['done'].astype(str)=='[]']
print(len(care_na))
del care_na['done']

In [ ]:
# regex_pattern_length=r'(?i)(professional upholstery)|(spot clean)|(vacuum)|(wipe clean)|(air dry)|(dry clean)|(tumble dry)||(washable)|(dishwasher safe)|(disposable)|(detergent safe)|(dry clean)|(gentle wash)|(hand wash)|(line dry)|(machine washable)|()'  
# care_na['matches'] = care_na['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
# td_care_na=care_na[care_na['matches'].astype(str)!='[]']
# td_care_na['matches']=td_care_na['matches'].apply(lambda x: x.capitalize())
# td_care_na

In [ ]:
print(len(td_care_na))
td_care_na

In [ ]:
# (?i)|(professional upholstery)|(.{0,20}removable.{0,20})|(spot.?clean)|(.{0,20}steam.{0,20})|(.{0,20}tumble.?dry.{0,20})|(vacuum)|(.{0,20}washable.{0,20})|(.{0,20}wipe.?clean.{0,20})|(air.?dry)|(.{0,20}dishwasher.{0,20})|(.{0,20}disposable.{0,20})|(.{0,20}detergent.{0,20})|(dry.?clean)|(.{0,20}gentle.{0,20})|(.{0,20}hand.?wash)|(.{0,20}line.?dry.{0,20})|(.{0,20}machine.?wash.{0,20})
regex_pattern_length=r'(?i)(.{0,20}tumble.?dry.{0,20})|(.{0,20}washable.{0,20})|(.{0,20}dishwasher.{0,20})|(.{0,20}disposable.{0,20})|(.{0,20}detergent.{0,20})|(dry.?clean)|(.{0,20}gentle.?wash.{0,20})|(.{0,20}hand.?wash)|(.{0,20}line.?dry.{0,20})|(.{0,20}machine.?wash.{0,20})|()'  
care_na['matches'] = care_na['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(care_na))
print(len(care_na[care_na['matches'].astype(str)=='[]']))
print(len(care_na[care_na['matches'].astype(str)!='[]']))

In [ ]:
care_na[care_na['matches'].astype(str)!='[]']

In [ ]:
no_match=care_na[care_na['matches'].astype(str)=='[]']
print(len(no_match[no_match['buckets'].astype(str)!='None']))
no_match[no_match['buckets'].astype(str)!='None'][500:1000]

In [ ]:
regex_pattern_length=r'(?i)(.{0,40}garden.?hose.{0,40})|()'  
no_match['matchez'] = no_match['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
no_match[no_match['matchez'].astype(str)!='[]']

In [ ]:
regex_pattern_length=r'(?i)(.{0,40}non.?skid.?pad.{0,40})|(.{0,40}assembly.?required.{0,40})|(.{0,40}garden.?hose.{0,40})|()'  
no_match['matchez'] = no_match['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(no_match[no_match['matchez'].astype(str)!='[]']))
no_match['matchez'].explode().value_counts()[0:500]

# Cooktop Compatibility

In [ ]:
# electric, glass top, gas, and induction.  

customer_id = '5'
formatted_attribute = 'Cooktop Compatibility'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
#           'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
cooktop = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(cooktop))
cooktop.head(2)

In [ ]:
regex_pattern_length=r'(?i)(all.?stove.?types)|(on all types of stove.?tops)|(on all stove.?tops)|(with all stove.?tops)|(for all stove.?tops)|(with all stove.?tops)|(will all stove.?tops)|(with all cooking surfaces)|(no matter what heat source)|(with all heati?n?g? sources)|(on any heat source)|(All range capable)|(on all heat sources)|()'  
cooktop['matches'] = cooktop['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
cooktop_final=cooktop[cooktop['matches'].astype(str)!='[]']
cooktop_final[curation_col]="Electric,Gas,Glass Top,Induction"
cooktop_final[curation_col]=cooktop_final[curation_col].apply(lambda x: str(f'["{x}"]')).apply(lambda x: re.sub(r',','","',str(x)))
matches_final=cooktop_final[['external_id',curation_col]]
print(len(matches_final))
# matches_final

In [ ]:
next_df=cooktop[cooktop['matches'].astype(str)=='[]']
regex_pattern_length=r'(?i)(electric.(?!.{0,20}kettle)(?!Casserole)(?!Wok)(?!Non.?Stick))|(glass.(?:(?:top)|(?:cook.?top)|(?:stove.?top))+)|(gas\b)|((?<!non)(?<!except.)induction)|(gas and electric stovetops)|(glass.?stove.?top)|()' 
next_df['matchez'] = next_df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
indiv=next_df[next_df['matchez'].astype(str)!='[]']
print(len(indiv))

indiv['matchez']=indiv['matchez'].apply(lambda x: str(x).title()).apply(lambda x: re.sub(r' ','',str(x))).apply(lambda x: re.sub(r"','",'","',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x)))                                                    
indiv['matchez']=indiv['matchez'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
indiv['matchez']=indiv['matchez'].apply(lambda x: sorted(x))
indiv['matchez']=indiv['matchez'].apply(lambda x: str(x).title()).apply(lambda x: re.sub(r' ','',str(x))).apply(lambda x: re.sub(r"','",'","',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"(?i)stove.?top|cooktop",' Top',str(x))).apply(lambda x: re.sub(r"(?i)glasstop",'Glass Top',str(x))).apply(lambda x: re.sub(r',","','","',str(x))).apply(lambda x: re.sub(r'\.','',str(x)))
indiv[curation_col]=indiv['matchez']

matches_indiv=indiv[['external_id',curation_col]]
matches_indiv.head(2)

In [ ]:
print(len(last))
last[curation_col]=''
match_last=last[['external_id',curation_col]]
# match_last

In [ ]:
print(len(next_df[(next_df['matchez'].astype(str)=='[]')&(next_df['matches'].astype(str)=='[]')]))
last=next_df[(next_df['matchez'].astype(str)=='[]')&(next_df['matches'].astype(str)=='[]')]
regex_pattern_length=r'(?i)(electric)|(glass)|(gas\b)|(induction)|(gas and electric stovetops)|(glass.?stove.?top)|()' 
last['matches'] = last['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
# last[last['matches'].astype(str)!='[]']

# Material

In [ ]:
customer_id = '5'
formatted_attribute = 'Material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_').replace('/','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
#           'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
material = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(material))
material.head(2)

In [ ]:
regex_pattern_length=r'(?i)(Polypropylene)|()' 
material['matches'] = material['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(material[material['matches'].astype(str)!='[]']))

In [ ]:
poly=material[material['buckets'].astype(str)=='Rugs & Mats B']
regex_pattern_length=r'(?i)(Polypropylene)|()' 
poly['matches'] = poly['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
poly[poly['matches'].astype(str)!='[]']

In [ ]:
polypropylene=poly[poly['matches'].astype(str)!='[]']
print(len(polypropylene))
polypropylene[curation_col]='Polypropylene'
match_poly=polypropylene[['external_id',curation_col]]
polypropylene[0:500]

In [ ]:
match_poly

In [ ]:
material=material[material['buckets'].astype(str)=='Prints, Posters, & Paintings']
material['value'].explode().value_counts()

In [ ]:
 material_an=material[material['value'].astype(str)=='n/a']
print(len(material_an))
pat_canvas='(?i)(canvas)|()'  
material_an['canvas'] = material_an['long_desc'].apply(lambda x: re_extract(pat_canvas, x))
material_an=material_an[material_an['canvas'].astype(str)!='[]']
# print(len(material_an))
# pat_indoorr='(?i)|(polystyrene)|(glass)|((?<!\/)wood\b.{0,10}frame(?!\/))|(paper\b)|()'  
# material_an['matches'] = material_an['long_desc'].apply(lambda x: re_extract(pat_indoorr, x))
# print(len(material_an[material_an['matches'].astype(str)=='[]']))
# material_an[material_an['matches'].astype(str)!='[]'][0:500]

In [ ]:
material_an[curation_col]='Canvas'
match_canvas=material_an[['external_id',curation_col]]
match_canvas

# Mount Type

In [ ]:
# customer_id = '5'
# formatted_attribute = 'Mount Type'
# attribute = formatted_attribute.lower().replace(' ','_').replace('-','_').replace('/','_')
# customer_name='%bedbathandbeyond%'

# params = {'customer_id': customer_id ,
#           'attribute': attribute,
# #           'buckets': str(buckets.split('\t'))[1:-1],
# #           'value':value,
#           'customer_name':customer_name
#          }
# curation_col = f'Q:{attribute}'
# mount = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
# print(len(mount))
# mount.head(2)

In [ ]:
# pat_indoorr='(?i)(freestanding)|()'  
# mount['matches'] = mount['value'].apply(lambda x: re_extract(pat_indoorr, x))
# mount_type=mount[mount['matches'].astype(str)!='[]']
# mount_type['buckets'].explode().value_counts()

In [ ]:
# print(len(mount_type[(mount_type['buckets'].astype(str)=='Shelves & Shelving Units')]))
# mount_type[(mount_type['buckets'].astype(str)=='Shelves & Shelving Units')][0:500]

# Light Filtration

In [ ]:
customer_id = '5'
formatted_attribute = 'Light Filtration'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_').replace('/','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
#           'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
light = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(light))
light.head(2)

In [ ]:
pat_bath='(?i)(.{0,40}blackout.{0,40})|()'  
light['matches'] = light['long_desc'].apply(lambda x: re_extract(pat_bath, x))
black=light[light['matches'].astype(str)!='[]']
blackout_df=black[black['value'].astype(str)=='Room Darkening']
blackout_df['value'].explode().value_counts()


In [ ]:
# blackout_df[curation_col]='["Blackout","Room Darkening"]'
# match_bo=blackout_df[['external_id',curation_col]]
# match_bo

In [ ]:
pat_bath='(?i)(.{0,40}Room.?Darkening.{0,40})|()'  
light['matches'] = light['long_desc'].apply(lambda x: re_extract(pat_bath, x))
room=light[light['matches'].astype(str)!='[]']
room_darkening=room[(room['value']!='Room Darkening')&(room['value']!='["Blackout","Room Darkening"]')&(room['value']!='["Blackout","Room Darkening","Sheer"]')]

In [ ]:
print(len(room_darkening))
blackout=room_darkening[room_darkening['value'].astype(str)=='Blackout']
na=room_darkening[room_darkening['value'].astype(str)=='n/a']
bo_lf=room_darkening[room_darkening['value'].astype(str)=='["Blackout","Light Filtering"]']
sheer=room_darkening[room_darkening['value'].astype(str)=='Sheer']
light_filtering=room_darkening[room_darkening['value'].astype(str)=='Light Filtering']

In [ ]:
# light_filtering[curation_col]='["Light Filtering","Room Darkening"]'
# match_lf=light_filtering[['external_id',curation_col]]
# match_lf

In [ ]:
# sheer[curation_col]='["Room Darkening","Sheer"]'
# match_sheer=sheer[['external_id',curation_col]]
# match_sheer

In [ ]:
# bo_lf[curation_col]='["Blackout","Light Filtering","Room Darkening"]'
# match_bolf=bo_lf[['external_id',curation_col]]
# match_bolf

In [ ]:
# blackout[curation_col]='["Blackout","Room Darkening"]'
# match_blackout=blackout[['external_id',curation_col]]
# match_blackout

In [ ]:
# na[curation_col]='Room Darkening'
# match_na=na[['external_id',curation_col]]
# match_na

# Indoor Outdoor

In [ ]:
customer_id = '5'
formatted_attribute = 'indoor_outdoor'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_').replace('/','_')
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
#           'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
indoor = query_from_file(file_name='../query/curated_products_VBOutDiamMM_buckets.sql', params=params)
print(len(indoor))
indoor.head(2)

In [ ]:
indoor['buckets'].explode().value_counts()

In [ ]:
# window=room[room['buckets'].astype(str)=='Window Blinds']
# print(len(window))
# window['value'].explode().value_counts()

In [ ]:
# print(len(window[window['value'].astype(str)=='Indoor & Outdoor']))
# indoor_outdoor=window[window['value'].astype(str)=='Indoor & Outdoor']
# # indoor_outdoor

In [ ]:
# outdoor=window[window['value'].astype(str)=='Outdoor']
# outdoor[curation_col]='n/a'
# match_outdoor=outdoor[['external_id',curation_col]]
# match_outdoor

In [ ]:
indoor_out=window[window['value'].astype(str)=='Indoor & Outdoor']
indoor_out[curation_col]='n/a'
match_in_out=indoor_out[['external_id',curation_col]]
# match_in_out

In [ ]:
regex_pattern_length=r'(?i)(water.?resistant)|(water.?proof)|()'  
indoor['water'] = indoor['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
print(len(indoor[indoor['water'].astype(str)!='[]']))
water=indoor[indoor['water'].astype(str)!='[]']

In [ ]:
water_idod=water[water['value'].astype(str)=='Indoor & Outdoor']
print(len(water_idod[(water_idod['buckets'].astype(str)=='Tablecloths & Table Runners')|(water_idod['buckets'].astype(str)=='Rugs & Mats')|(water_idod['buckets'].astype(str)=='Rugs & Mats B')]))
tc_rm=water_idod[(water_idod['buckets'].astype(str)=='Tablecloths & Table Runners')|(water_idod['buckets'].astype(str)=='Rugs & Mats')]
print(len(water_idod[(water_idod['buckets'].astype(str)!='Tablecloths & Table Runners')&(water_idod['buckets'].astype(str)!='Rugs & Mats')]))
# water_idod[(water_idod['buckets'].astype(str)!='Tablecloths & Table Runners')&(water_idod['buckets'].astype(str)!='Rugs & Mats')]

In [ ]:
water_na=water[water['value'].astype(str)=='n/a']
curtain=water_na[water_na['buckets'].astype(str)=='Curtains']
regex_pattern_length=r'(?i)(shower)|(bath)|()'  
curtain['curtain'] = curtain['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
curtain[curtain['curtain'].astype(str)=='[]']
# curtain
water_office=water[water['buckets'].astype(str)=='Office Chairs']
print(len(water_office))
# water_office

In [ ]:
print(len(water_idod[(water_idod['buckets'].astype(str)=='Tablecloths & Table Runners')|(water_idod['buckets'].astype(str)=='Rugs & Mats')|(water_idod['buckets'].astype(str)=='Rugs & Mats B')]))
rugs=water[(water['buckets'].astype(str)=='Tablecloths & Table Runners')|(water['buckets'].astype(str)=='Rugs & Mats')|(water['buckets'].astype(str)=='Rugs & Mats B')]
regex_pattern_length=r'(?i)(indoor.{0,10}outdoor)|(table.?cloth)|()'  
rugs['indoor'] = rugs['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_area=r'(?i)(area.?rug)|(runner)|(kitchen)|(bath)|(anti.?fatigue)|(accent)|(shower)|()'  
# rugs['area'] = rugs['name'].apply(lambda x: re_extract(regex_pattern_area, x))
# rugs=rugs[(rugs['area'].astype(str)=='[]')]
print(len(rugs[(rugs['indoor'].astype(str)=='[]')&(rugs['value'].astype(str)!='Outdoor')]))
# rugs[(rugs['indoor'].astype(str)=='[]')&(rugs['value'].astype(str)!='Outdoor')]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, match_throw_na)